# Mission to Mars (Data Scraping)

In [1]:
# All scraping and analysis
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
import requests
import time

In [2]:
#url list
url_mars_news = 'https://mars.nasa.gov/news/'
url_jpl_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url_mars_weather = 'https://twitter.com/marswxreport?lang=en'
url_mars_facts = 'https://space-facts.com/mars/'
url_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


## Mars News

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

browser.visit(url_mars_news)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_title = soup.find('div', class_="list_text").find('div', class_="content_title").get_text(strip=True)
news_p = soup.find('div', class_="article_teaser_body").get_text(strip=True)

print(news_title)
print('-------------------')
print(news_p)

browser.quit()

NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light
-------------------
Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.


## JPL Featured Space Image

In [4]:
image_root = 'https://www.jpl.nasa.gov'

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

browser.visit(url_jpl_image)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

jpg_path = soup.find('article', class_="carousel_item")["style"]

# print(jpg_path)
# print(jpg_path.find('url'))
# print(jpg_path[23:])
featured_image_url = image_root + jpg_path[jpg_path.find('url')+5:jpg_path.find('jpg')+3]
# print(jpg_path[jpg_path.find('url')+5:jpg_path.find('jpg')+3])
print(featured_image_url)

browser.quit()

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16613-1920x1200.jpg


## Weather

In [21]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

browser.visit(url_mars_weather)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

weather_tweet = soup.find('div', attrs = {'class': 'css-901oao', 'lang': 'en', 'dir':'auto'}).get_text(strip=True)

browser.quit()



InSight sol 604 (2020-08-07) low -91.7ºC (-133.1ºF) high -16.3ºC (2.6ºF)
winds from the WNW at 8.1 m/s (18.2 mph) gusting to 20.2 m/s (45.2 mph)
pressure at 7.90 hPa


## Facts

In [27]:

tables = pd.read_html(url_mars_facts)
mars_facts_df = tables[0]
mars_facts_df.columns = ['attribute', 'value']
mars_facts_df.set_index('attribute', inplace=True)
# mars_facts_df
facts_html = mars_facts_df.to_html()
facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>attribute</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n 

## Hemisphere Photos